In [ ]:
!pip install libigl matplotlib numpy polyscope gpytoolbox

## Introduction to Mesh Parameterization: Follow-Along Coding 
In this notebook we will run through some of the computations that you will be asked to perform in your exercises, on a simpler set of meshes. 


We will first demonstrate how to perform a trivial (triangle soup) parameterization on the simplest mesh possible -- a single triangle, and verify that the trivial parameterization is in fact distortionless. You will be expected to know how to scale this trivial parameterization to arbitrary meshes for assignment 103. 


By the time you run through this notebook, you should have a clearer understanding of how to set up and solve the linear system for computing the Tutte embedding, as well as how to alter the system for different choices of Laplacian weights. 


Furthermore, you should know how to compute the Jacobian of the parameterization, obtain the singular values, measure the different types of distortion, and visualize them. 

### Part 1: Single Triangle Parameterization ###
In the case we want to obtain a perfect parameterization of every triangle in the mesh, and we don't care about cuts, then we can always use the trivial parameterization, which projects each vertex of a triangle onto its local basis (see figure below). 

![image](local_basis.png)

In the above image $x_i$ is set to the origin of the local basis, and the triangle edge $x_j - x_i$ is chosen as the first basis vector $X$. From there, the normal is computed as the cross product between the two triangle edges connected to $x_i$, and another cross product is taken to get the second basis vector $Y$. 

After these local (2D) bases are defined, we can find the coordinates of each triangle vertex in terms of these bases using the dot product. Specifically, we take the dot product between the edge vectors of each vertex with $X$, $Y$ respectively to get the local coordinates $(u, v)$, which are the 2D parameterization coordinates. The parameterization coordinates for the triangle in the figure are explicitly written out below. 

$$\begin{align*}
u_i = (0, 0) \text{ since } v_i \text{ is the origin in the local basis} \\
u_j = ((x_j - x_i) \cdot X, (x_j - x_i) \cdot Y) \\
u_k = ((x_k - x_i) \cdot X, (x_k - x_i) \cdot Y) \\
\end{align*}$$

In [2]:
import numpy as np

# This function computes a trivial parameterization of a single triangle using the technique described above.
def trivial_parameterization(triangle):
    # Local basis origin is the first triangle vertex
    origin = triangle[0]

    # Get the normal of the plane defined by the triangle
    e1 = triangle[1] - origin
    e2 = triangle[2] - origin
    normal = np.cross(e1, e2)
    normal /= np.linalg.norm(normal)

    # We will use e1 as the first basis. Second basis will be cross between e1 and the normal
    basis1 = e1 / np.linalg.norm(e1)
    basis2 = np.cross(normal, basis1)
    basis2 /= np.linalg.norm(basis2)

    # First vertex is the origin. Project the other two vertices onto the local bases.
    uv0 = np.array([0,0])
    uv1 = np.array([np.dot(e1, basis1), np.dot(e1, basis2)])
    uv2 = np.array([np.dot(e2, basis1), np.dot(e2, basis2)])

    return np.array([uv0, uv1, uv2]), origin, basis1, basis2

In [4]:
# Define a triangle and compute its trivial parameterization
triangle = np.array([[5.1, 2.8, 9.9],
                     [-1, 8, 2.3],
                     [8, 4.4, 6.7]])

uvs, origin, basis1, basis2 = trivial_parameterization(triangle)

The below polyscope code visualizes the original 3D triangle (blue) with the local bases $X$ (red) and $Y$ (blue) plotted. The parameterization triangle (green) is also plotted. 

In [6]:
# Visualize the bases and the UV coordinates
import polyscope as ps

ps.init()
ps.remove_all_structures()
ps.register_surface_mesh("single tri", triangle, np.array([[0,1,2]]), edge_width=1, color = (0.1, 0.2, 1))
ps.register_curve_network("basis1", np.array([origin, origin + basis1]), np.array([[0,1]]), radius=0.01, color=(1, 0, 0))
ps.register_curve_network("basis2", np.array([origin, origin + basis2]), np.array([[0,1]]), radius=0.01, color=(0, 0, 1))
ps.register_surface_mesh("uv tri", uvs, np.array([[0,1,2]]), edge_width=1, color = (0.2, 1, 0.2))
ps.reset_camera_to_home_view()
ps.show()

Let's confirm that the trivial parameterization is indeed distortionless by finding the Jacobian of the mapping. 

This is as simple as getting the gradient operator of this triangle, reshaping a bit, and multiplying it against the parameterization values. 

In [ ]:
def get_jacobian(vs, fs, uvmap):
    """ Get jacobian of mesh given an input UV map

    Args:
        vs (np.ndarray): V x 3 array of vertex positions
        fs (np.ndarray): F x 3 integer array of face indices
        uvmap (np.ndarray): V x 2 array of UV coordinates

    Returns:
        _type_: _description_
    """
    # Visualize distortion
    from igl import grad
    G = np.array(grad(vs, fs).todense())

    # NOTE: currently gradient is organized as X1, X2, X3, ... Y1, Y2, Y3, ... Z1, Z2, Z3 ... reshape to X1, Y1, Z1, ...
    splitind = G.shape[0]//3
    newG = np.zeros_like(G) # F*3 x V
    newG[::3] = G[:splitind]
    newG[1::3] = G[splitind:2*splitind]
    newG[2::3] = G[2*splitind:]

    J = (newG @ uvmap).reshape(-1, 3, 2) # F x 3 x 2
    return J

J = get_jacobian(triangle, np.array([[0,1,2]]), uvs)

![image](ss.png)

We can use numpy to obtain the singular values of this map. Recall that each face of the mesh gets two singular values, with the distortion measures defined above. 

In [17]:
_, S, _ = np.linalg.svd(J)

S = S[0] # Get singular values of first (only) face

## Let's confirm there is no distortion
# It is obvious if we just print the values
print("##### Singular values of the trivial parameterization #####")
print(S)

# Area preservation: s1 * s2 == 1
assert np.isclose(S[0] * S[1], 1)

# Conformal (Angle) preservation: s1 == s2
assert np.isclose(S[0], S[1])

# Isometric distortion: s1 == s2 == 1
assert np.isclose(S[0], 1)
assert np.isclose(S[1], 1)

##### Singular values of the trivial parameterization #####
[1. 1.]


### Part 2: Fixed Boundary Parameterization of Simple Pyramid ###
Now let's take a look at a slightly more difficult example -- an open pyramid (boundary at the base). We will compute fixed boundary parameterizations over this mesh using different choices of Laplacian weights (explained in the writeup). 

The code below will show how to setup the system of equations to be solved in matrix form, which will allow us to make use of standard Python libraries to solve them. It will also demonstrate how to recompute the parameterization using the same matrices with different sets of weights. Please refer to the exercise writeup to see how the matrix form of this linear system is derived. 

We will investigate how different choices of weights affects both the resulting distortion and the injectivity of the parameterization. 

In [20]:
### Define the pyramid and visualize
pyramid_vs = np.array([[3, 0, 1],
                    [-1, -1, 0],
                    [1, -1, 0],
                    [1, 1, 0],
                    [-1, 1, 0]], dtype=np.float32)

pyramid_fs = np.array([[0, 3, 2],
                       [0, 4, 3],
                       [0, 1, 2],
                       [0, 4, 1]], dtype=int)

## Rotate the mesh around to inspect it!
ps.init()
ps.remove_all_structures()
ps.register_surface_mesh("pyramid", pyramid_vs, pyramid_fs, edge_width=1)

# The boundary points are indicated with red spheres
boundaryvs = np.array([pyramid_vs[1], pyramid_vs[2], pyramid_vs[3], pyramid_vs[4]])
boundaryes = np.array([[0,0], [1,1], [2,2], [3,3]])
ps.register_curve_network("boundaries", boundaryvs, boundaryes, radius=0.01, color=(1, 0, 0))

ps.reset_camera_to_home_view()
ps.show()

The figure of the pyramid indicates there is an obvious choice of boundary positions we can set. For each of the 4 boundary vertices, we can simply drop the z-coordinate. Once we set the boundary positions, we can proceed to solving the Tutte parameterization problem. Recall that with the Tutte parameterization, all Laplacian (edge) weights are set to 1 ($w_{ij} in the below formula). 

$$\min_{\mathbf{U}} \sum_{\{i,j\} \in \mathbf{E}} w_{ij} || \mathbf{u}_i - \mathbf{u}_j||^2$$

In [40]:
### Set the boundary vertex positions
boundary_idxs = [1, 2, 3, 4]
boundary_positions = pyramid_vs[1:, :2]

### The below code generates the matrices for the Tutte linear system
def setup_tutte_matrices(vertices, faces, boundary_idxs):
    """ Set up the Tutte linear system (laplacian weights of 1)

    vertices (np.ndarray): V x 3 array of vertex positions
    faces (np.ndarray): F x 3 integer array of face indices
    boundary_idxs (np.ndarray): B array of boundary vertex indices
    boundary_positions (np.ndarray): B x 2 array of boundary vertex positions

    Returns:
        L (np.ndarray): V x V Laplacian matrix with boundary values set to 0
        Lb (np.ndarray): V x B boundary Laplacian
    """

    # Initialize the laplacian matrix
    L = np.zeros((vertices.shape[0], vertices.shape[0]))

    # Get edge array
    edges = np.concatenate((faces[:, [0, 1]], faces[:, [1, 2]], faces[:, [2, 0]]), axis=0)

    L[edges] = 1

    # Off-diagonal elements are negative and diagonal is the negative sum of the row
    L = np.diag(np.sum(L, axis=1)) - L

    Lb = L[:, boundary_idxs]

    # Set the off-diagonal boundary columns to 0
    Ldiag = np.diag(L).copy()

    print(Ldiag)

    L[:, boundary_idxs] = 0
    np.fill_diagonal(L, Ldiag)

    return L, Lb

L, Lb = setup_tutte_matrices(pyramid_vs, pyramid_fs, boundary_idxs)

### Solve the Tutte linear system
tutte_solution = np.linalg.solve(L, Lb @ boundary_positions)

# Final solution is combining the Tutte solution with the boundary positions
tutte_uv = np.zeros((pyramid_vs.shape[0], 2))
tutte_uv[boundary_idxs] = boundary_positions
solution_idxs = np.setdiff1d(np.arange(pyramid_vs.shape[0]), boundary_idxs)
tutte_uv[solution_idxs] = tutte_solution[solution_idxs]

[4. 4. 4. 4. 4.]


We can now visualize the final parameterization using Polyscope. What do you notice about the UV position of the single non-pinned vertex? Is the parameterization injective (do any triangles overlap)? Is this result expected based on what you learned about the choice of weights? 

In [ ]:
# Visualize the embedding
ps.init()
ps.remove_all_structures()
ps_pyramid = ps.register_surface_mesh("pyramid", pyramid_vs, pyramid_fs, edge_width=1, enabled=True)

# Color the faces to get the correspondence
fcolors = np.arange(pyramid_fs.shape[0])
ps_pyramid.add_scalar_quantity("faces", fcolors, defined_on='faces', cmap='viridis', enabled=True)

# The boundary points are indicated with red spheres
boundaryvs = np.array([pyramid_vs[1], pyramid_vs[2], pyramid_vs[3], pyramid_vs[4]])
boundaryes = np.array([[0,0], [1,1], [2,2], [3,3]])
ps.register_curve_network("boundaries", boundaryvs, boundaryes, radius=0.01, color=(1, 0, 0), enabled=True)

ps_tutte = ps.register_surface_mesh("tutte parameterization", tutte_uv + np.array([[0, 3]]), pyramid_fs, edge_width=1, enabled=True)
ps_tutte.add_scalar_quantity("faces", fcolors, defined_on='faces', cmap='viridis', enabled=True)

ps.reset_camera_to_home_view()
ps.show()

Let's compute and visualize the distortion of the Tutte embedding.

In [ ]:
## TODO:

Recall the important connection we learned between choice of weights and the injectivity of the parameterization! 

**If the pinned bounary is convex and the weights are positive, the the solution to the linear system will be injective.**

Now let's see the effect of a few other choices of weights. 

Now try your hand at computing the parameterization with your own choice of weights! What do you observe? 